In [1]:
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, Dense, Activation
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import numpy as np
from six.moves import range
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 60000
SPLIT_RATIO = 0.8
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
OUTPUTLEN = DIGITS * 2
chars = '0123456789* '
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

# Data Generation

In [6]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}*{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    ans += ' ' * (OUTPUTLEN - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 60000


In [7]:
print(questions[:5], expected[:5])

['31*9   ', '888*5  ', '85*1   ', '70*223 ', '206*258'] ['279   ', '4440  ', '85    ', '15610 ', '53148 ']


# Processing

In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), OUTPUTLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, OUTPUTLEN)

Vectorization...


In [9]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

idx = int(SPLIT_RATIO * TRAINING_SIZE)

# train_test_split
train_x = x[:idx]
train_y = y[:idx]
test_x = x[idx:]
test_y = y[idx:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(43200, 7, 12)
(43200, 6, 12)
Validation Data:
(4800, 7, 12)
(4800, 6, 12)
Testing Data:
(12000, 7, 12)
(12000, 6, 12)


In [10]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False  True False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False  True False False
   False]
  [False False False False False False False False False False False
    True]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False Fal

# Build Model

In [11]:
print('Build model...')

def build_model():
    input_shape = (MAXLEN, len(chars))

    model = Sequential()

    # Encoder:
    model.add(Bidirectional(LSTM(20), input_shape=input_shape))
    model.add(BatchNormalization())

    # The RepeatVector-layer repeats the input n times
    model.add(RepeatVector(OUTPUTLEN))

    # Decoder:
    model.add(Bidirectional(LSTM(20, return_sequences=True)))
    model.add(BatchNormalization())

    model.add(TimeDistributed(Dense(len(chars))))
    model.add(Activation('softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.01),
        metrics=['accuracy'],
    )

    return model

model = build_model()
print(model.summary())

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 40)                5280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 40)                160       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 6, 40)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 6, 40)             9760      
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 40)             160       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 6, 12)             492       
_______________________________________________________

# Training

In [12]:
for iteration in range(50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    history = model.fit(x_train, y_train,
                        batch_size=BATCH_SIZE,
                        epochs=1,
                        validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 8s 196us/step - loss: 1.6794 - acc: 0.3779 - val_loss: 2.0007 - val_acc: 0.3715
Q 883*59  T 52097  ☒ 44425 
Q 3*992   T 2976   ☒ 444   
Q 4*366   T 1464   ☒ 122   
Q 802*188 T 150776 ☒ 155980
Q 625*5   T 3125   ☒ 455   
Q 512*630 T 322560 ☒ 434200
Q 553*21  T 11613  ☒ 12355 
Q 916*5   T 4580   ☒ 4500  
Q 5*72    T 360    ☒ 40    
Q 19*200  T 3800   ☒ 4000  

--------------------------------------------------
Iteration 1
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 5s 120us/step - loss: 1.3335 - acc: 0.4659 - val_loss: 1.3228 - val_acc: 0.4676
Q 807*29  T 23403  ☒ 21323 
Q 8*993   T 7944   ☒ 7112  
Q 22*732  T 16104  ☒ 18116 
Q 63*330  

43200/43200 [==============================] - 5s 118us/step - loss: 0.9777 - acc: 0.6215 - val_loss: 0.9366 - val_acc: 0.6368
Q 255*499 T 127245 ☒ 137155
Q 11*508  T 5588   ☒ 5758  
Q 124*9   T 1116   ☑ 1116  
Q 8*244   T 1952   ☒ 1922  
Q 482*497 T 239554 ☒ 281124
Q 292*72  T 21024  ☒ 21014 
Q 891*961 T 856251 ☒ 896151
Q 4*824   T 3296   ☒ 3266  
Q 757*151 T 114307 ☒ 119117
Q 883*59  T 52097  ☒ 56557 

--------------------------------------------------
Iteration 13
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 5s 119us/step - loss: 0.9656 - acc: 0.6267 - val_loss: 0.9372 - val_acc: 0.6334
Q 578*88  T 50864  ☒ 49444 
Q 2*773   T 1546   ☒ 1516  
Q 962*61  T 58682  ☒ 56372 
Q 936*364 T 340704 ☒ 344444
Q 85*66   T 5610   ☑ 5610  
Q 121*99  T 11979  ☒ 12949 
Q 84*195  T 16380  ☒ 16490 
Q 56*769  T 43064  ☒ 42464 
Q 33*712  T 23496  ☒ 22416 
Q 371*47  T 17437  ☒ 16237 

--------------------------------------------------
Iteration 

Q 40*638  T 25520  ☒ 25820 
Q 143*36  T 5148   ☒ 5508  
Q 96*932  T 89472  ☒ 93332 
Q 897*18  T 16146  ☒ 16506 
Q 48*865  T 41520  ☒ 41060 
Q 925*65  T 60125  ☒ 61525 
Q 172*88  T 15136  ☒ 15256 
Q 619*200 T 123800 ☒ 121800
Q 246*17  T 4182   ☒ 3102  
Q 38*11   T 418    ☒ 488   

--------------------------------------------------
Iteration 26
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 5s 118us/step - loss: 0.8618 - acc: 0.6640 - val_loss: 0.8275 - val_acc: 0.6772
Q 431*272 T 117232 ☒ 115072
Q 22*375  T 8250   ☒ 8350  
Q 56*956  T 53536  ☒ 52976 
Q 6*551   T 3306   ☒ 3336  
Q 59*43   T 2537   ☒ 2497  
Q 16*516  T 8256   ☒ 8336  
Q 66*599  T 39534  ☒ 39444 
Q 518*88  T 45584  ☒ 44444 
Q 37*362  T 13394  ☒ 12444 
Q 681*46  T 31326  ☒ 30456 

--------------------------------------------------
Iteration 27
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 5s 116us/step - lo

Q 70*52   T 3640   ☑ 3640  
Q 374*528 T 197472 ☒ 201772
Q 82*75   T 6150   ☑ 6150  
Q 14*916  T 12824  ☒ 13264 
Q 48*254  T 12192  ☒ 12112 
Q 947*52  T 49244  ☒ 49224 

--------------------------------------------------
Iteration 39
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 5s 121us/step - loss: 0.8166 - acc: 0.6798 - val_loss: 0.7887 - val_acc: 0.6878
Q 260*41  T 10660  ☑ 10660 
Q 129*272 T 35088  ☒ 33668 
Q 127*587 T 74549  ☒ 72229 
Q 778*1   T 778    ☒ 768   
Q 487*340 T 165580 ☒ 168280
Q 67*940  T 62980  ☒ 62580 
Q 926*2   T 1852   ☒ 1832  
Q 407*18  T 7326   ☒ 7226  
Q 96*268  T 25728  ☒ 25428 
Q 196*113 T 22148  ☒ 20568 

--------------------------------------------------
Iteration 40
Train on 43200 samples, validate on 4800 samples
Epoch 1/1
43200/43200 [==============================] - 5s 118us/step - loss: 0.8082 - acc: 0.6831 - val_loss: 0.7810 - val_acc: 0.6904
Q 17*144  T 2448   ☒ 2238  
Q 164*20  T 3280   ☒ 3

# Testing

In [13]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
    

MSG : Prediction


In [14]:
pred = model.predict_classes(test_x)

In [15]:
prediction = np.apply_along_axis(ctable.decode, 1, pred, False)

In [16]:
answer = np.apply_along_axis(lambda x: "".join(ctable.indices_char[i] for i in x), 1, test_y.argmax(axis=-1))

In [17]:
np.sum(prediction == answer) / answer.shape[0]

0.1185